# OVERVIEW

# There are 3 Main Process that will be applied in this case :

1. Data Preparation and Preprocessing
2. Modelling Comparison (7 different types of models, see which is the most sensitive and accurate)
3. Hyperparameter Tuning
4. Metrics Evaluation for the fixed Model


# Data Preparation and Preprocessing

# About The Data

In total, I downloaded 11 seperate datasets with around a million records each. The datasets can be found at 
https://github.com/yingjia-liu/Msft-Grab-FRS/blob/master/Safety/Safety_Problem_Statement.md. 

A https://msftgrab.z23.web.core.windows.net/safety/Safety_DataSet_Aggregated.zip (full aggregated dataframe) was available, but is corrupted upon opening, hence requiring manual aggregation of singulaer datasets.



In This Section, We have 11 data sources. There are :
1. 10 driver datasets 
2. Label datasets

Steps of our data processing are :
1. Import all datasets
2. Concate 10 driver datasets to become a single main dataset as dataframe
3. Get Dataset Information (Like some missing value, size of table, metadata, etc)
4. Do some Feature Engineering that adds new features to the dataset
5. Aggregate the dataset by 'bookingID' variable with mean (average)
6. Merge 'label' dataset with the main dataset
7. Clean the Dataset
    1. Avoid redundant values
    2. Missing Value Checking


# Data Preparation Starts From Here

# Importing All Modules (Pre-Step 1)

In [1]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
# GridSearchCV is like k-fold.They work together,training and testing sets (70/30)
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold
import numpy as np
# Does all mathematical operations. So, it does ALL feature engineering as well.
import pandas as pd
# For the dataframe
import matplotlib.pyplot as plt
# For plotting charts
import seaborn as sns
# For plotting charts
from sklearn.metrics import classification_report
# Brief reporting tool for classification. See classification report below
from imblearn.pipeline import Pipeline
# More efficient than GridSearch CV
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import make_scorer,accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,log_loss
warnings.filterwarnings("ignore")

# Data Preparation

# Importing All Datasets (Step 1)

### Load the datasets

In [2]:
data1 = pd.read_csv('part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data2 = pd.read_csv('part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data3 = pd.read_csv('part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data4 = pd.read_csv('part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data5 = pd.read_csv('part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data6 = pd.read_csv('part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data7 = pd.read_csv('part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data8 = pd.read_csv('part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data9 = pd.read_csv('part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
data10 = pd.read_csv('part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv')
label = pd.read_csv('part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv')

# Concatenating All Datasets (Step 2)

In [3]:
data=pd.concat([data1,data2,data3,data4,data5,data6,data7,data8,data9,data10],0)
data.info()
# data.head is useful for quickly testing if your object has the right type of data in it.
# data.info prints information about a DataFrame including the index dtype and columns, non-null values and memory usage

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16135561 entries, 0 to 1613561
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   bookingID       int64  
 1   Accuracy        float64
 2   Bearing         float64
 3   acceleration_x  float64
 4   acceleration_y  float64
 5   acceleration_z  float64
 6   gyro_x          float64
 7   gyro_y          float64
 8   gyro_z          float64
 9   second          float64
 10  Speed           float64
dtypes: float64(10), int64(1)
memory usage: 1.4 GB


In [4]:
data.head()
# data.head is useful for quickly testing if your object has the right type of data in it. Same as before, but this is after the concat and nicer formatting

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,Speed
0,1202590843006,3.000,353.0,1.228867,8.900100,3.986968,0.008221,0.002269,-0.009966,1362.0,0.000000
1,274877907034,9.293,17.0,0.032775,8.659933,4.737300,0.024629,0.004028,-0.010858,257.0,0.190000
2,884763263056,3.000,189.0,1.139675,9.545974,1.951334,-0.006899,-0.015080,0.001122,973.0,0.667059
3,1073741824054,3.900,126.0,3.871543,10.386364,-0.136474,0.001344,-0.339601,-0.017956,902.0,7.913285
4,1056561954943,3.900,50.0,-0.112882,10.550960,-1.560110,0.130568,-0.061697,0.161530,820.0,20.419409


## Feature Engineering

### Feature Distance

In [5]:
###Distance
data['distance']=data['Speed']*data['second']

### Feature Speed

To make this feature, I multiply 'acceleration' with 'second' to produce feature 'second'.
I use Euclid distance to determine the interaction from the Feature Speed. 


I use this method to add new faeture because this feature contain direction/vector like (x,y,z)
Euclidean distance between two points in Euclidean space is a number, the length of a line segment between the two points.

(This is basically just replicating Euclid distance formula. Nothing revolutionary here.See the formula)

In [6]:
###Speed
data['speed_x']=data['acceleration_x']*data['second']
data['speed_y']=data['acceleration_y']*data['second']
data['speed_z']=data['acceleration_z']*data['second']
data['speed_xy']=np.sqrt(data['speed_x']**2+data['speed_y']**2)
data['speed_xz']=np.sqrt(data['speed_x']**2+data['speed_z']**2)
data['speed_yz']=np.sqrt(data['speed_z']**2+data['speed_y']**2)
data['speed_xyz']=np.sqrt(data['speed_x']**2+data['speed_y']**2+data['speed_z']**2)

### Feature Radian

To make this feature, I multiply 'gyro' with 'second' to produce feature 'second'
I use Euclid distance to determine the interaction from the Feature Radian. 
I use this method to add new faeture because this feature contain direction/vector like (x,y,z)

Same deal as before (speed)

In [7]:
###radian
data['rad_x']=data['gyro_x']*data['second']
data['rad_y']=data['gyro_y']*data['second']
data['rad_z']=data['gyro_z']*data['second']
data['rad_xy']=np.sqrt(data['rad_x']**2+data['rad_y']**2)
data['rad_xz']=np.sqrt(data['rad_x']**2+data['rad_z']**2)
data['rad_yz']=np.sqrt(data['rad_z']**2+data['rad_y']**2)
data['rad_xyz']=np.sqrt(data['rad_x']**2+data['rad_y']**2+data['rad_z']**2)

### Feature Acceleration

I make the interaction variable with all the combination of the feature 'Acceleration' based on the vector theorem. 
I use Euclid Method to combine the feature

In [8]:
###Acceleration
data['acc_xy']=np.sqrt(data['acceleration_x']**2+data['acceleration_y']**2)
data['acc_xz']=np.sqrt(data['acceleration_x']**2+data['acceleration_z']**2)
data['acc_yz']=np.sqrt(data['acceleration_z']**2+data['acceleration_y']**2)
data['acc_xyz']=np.sqrt(data['acceleration_x']**2+data['acceleration_y']**2+data['acceleration_z']**2)

### Feature Gyro

I make the interaction variable with all the combination of the feature 'Gyro' based on the vector theorem. 
I use Euclid Method to combine the feature

In [9]:
###Gyro
data['gyro_xy']=np.sqrt(data['gyro_x']**2+data['gyro_y']**2)
data['gyro_xz']=np.sqrt(data['gyro_x']**2+data['gyro_z']**2)
data['gyro_yz']=np.sqrt(data['gyro_z']**2+data['gyro_y']**2)
data['gyro_xyz']=np.sqrt(data['gyro_x']**2+data['gyro_y']**2+data['gyro_z']**2)

In [10]:
###Interaction
data['acc_gyro_x']=data['acceleration_x']*data['gyro_x']
data['acc_gyro_y']=data['acceleration_y']*data['gyro_y']
data['acc_gyro_z']=data['acceleration_z']*data['gyro_z']
data['acc_gyro_xy']=np.sqrt(data['acc_gyro_x']**2+data['acc_gyro_y']**2)
data['acc_gyro_xz']=np.sqrt(data['acc_gyro_x']**2+data['acc_gyro_z']**2)
data['acc_gyro_yz']=np.sqrt(data['acc_gyro_z']**2+data['acc_gyro_y']**2)
data['acc_gyro_xyz']=np.sqrt(data['acc_gyro_x']**2+data['acc_gyro_y']**2+data['acc_gyro_z']**2)

### Feature Count

I aggregate all the 'bookingID' Feature by count so we know how many calls the driver got.
Also known as "how many times this bookingID appears"

Eg: We have booking ID 101. How many times does this appear?

In [11]:
a=data['bookingID'].value_counts()
ID=pd.DataFrame(a)
ID['id']=ID.index
ID['count']=ID['bookingID']
#ID.head()
Count_df=pd.DataFrame()
# Convert it into dataframe and call it ID. BookingID = count (ID)
Count_df['bookingID']=ID['id']
Count_df['count']=ID['count']

In [12]:
Count_df
# Shows you the count of booking ID

,bookingID,count
438086664371,438086664371,7561
1374389534819,1374389534819,4499
34359738469,34359738469,4302
1108101562533,1108101562533,3925
747324309632,747324309632,3674
...,...,...
1537598292022,1537598292022,120
180388626478,180388626478,120
317827579936,317827579936,120
472446402608,472446402608,120


### Data Aggregation by Mean for BookingID and Merge the aggregation data with label

In [13]:
variable=['Accuracy', 'Bearing', 'acceleration_x', 'acceleration_y',
       'acceleration_z', 'gyro_x', 'gyro_y', 'gyro_z', 'second', 'Speed',
       'distance', 'speed_x', 'speed_y', 'speed_z', 'speed_xy', 'speed_xz',
       'speed_yz', 'speed_xyz', 'rad_x', 'rad_y', 'rad_z', 'rad_xy', 'rad_xz',
       'rad_yz', 'rad_xyz', 'acc_xy', 'acc_xz', 'acc_yz', 'acc_xyz', 'gyro_xy',
       'gyro_xz', 'gyro_yz', 'gyro_xyz', 'acc_gyro_x', 'acc_gyro_y',
        'acc_gyro_z', 'acc_gyro_xy', 'acc_gyro_xz', 'acc_gyro_yz']

In [14]:
dataAg=data.groupby('bookingID', as_index=False)[variable].mean()
# Note: This takes all the grouped by bookingID values and gets the mean (average) of them, so you don't end up with loads of values.
dataAg.info()
df_main = pd.merge(dataAg,label,how='left',left_on = 'bookingID',right_on = 'bookingID')

print(df_main.info())
df_main.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 40 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bookingID       20000 non-null  int64  
 1   Accuracy        20000 non-null  float64
 2   Bearing         20000 non-null  float64
 3   acceleration_x  20000 non-null  float64
 4   acceleration_y  20000 non-null  float64
 5   acceleration_z  20000 non-null  float64
 6   gyro_x          20000 non-null  float64
 7   gyro_y          20000 non-null  float64
 8   gyro_z          20000 non-null  float64
 9   second          20000 non-null  float64
 10  Speed           20000 non-null  float64
 11  distance        20000 non-null  float64
 12  speed_x         20000 non-null  float64
 13  speed_y         20000 non-null  float64
 14  speed_z         20000 non-null  float64
 15  speed_xy        20000 non-null  float64
 16  speed_xz        20000 non-null  float64
 17  speed_yz        20000 non-null 

,bookingID,Accuracy,Bearing,acceleration_x,acceleration_y,acceleration_z,gyro_x,gyro_y,gyro_z,second,...,gyro_xz,gyro_yz,gyro_xyz,acc_gyro_x,acc_gyro_y,acc_gyro_z,acc_gyro_xy,acc_gyro_xz,acc_gyro_yz,label
0,0,10.165339,176.526099,-0.711264,-9.613822,-1.619658,0.003328,-0.006118,-0.004188,903.526892,...,0.068242,0.082938,0.100772,-0.011891,0.067631,0.001921,0.633826,0.105596,0.645055,0
1,1,3.718763,124.198590,-0.525406,9.532086,-2.198999,-0.002467,-0.007540,0.000405,581.175088,...,0.032787,0.060189,0.066187,0.002711,-0.070063,-0.001414,0.497354,0.056595,0.503368,1
2,2,3.930626,173.794872,0.306786,9.843183,0.139347,0.006458,-0.012861,0.002597,339.441026,...,0.041984,0.082883,0.097433,-0.000493,-0.124528,-0.003984,0.718235,0.038764,0.716790,1
3,4,10.000000,151.807013,-0.365117,-9.406439,-2.613639,-0.022884,0.023232,-0.000376,547.495430,...,0.062106,0.098309,0.108875,0.010705,-0.224052,0.000394,0.743528,0.125491,0.767880,1
4,6,4.586721,197.812785,0.490616,9.538043,2.355059,0.003877,0.000436,0.002930,547.000000,...,0.057885,0.073102,0.089589,0.002100,0.007564,0.005758,0.549155,0.100458,0.560269,0


# Model Comparison

### Preparing data for modelling

In [15]:
X = df_main.drop(['label'],1)
df_main = df_main.drop(['bookingID'],axis=1)
df_main = pd.get_dummies(df_main)
# All the dataAg are in X. It just drops the label column from df.main, and the label column goes to y.
# get_dummies will convert all the variables into binary (0/1) for Machine Learning
y = df_main['label']

### Check the data balance

In [16]:
df_main['label'].value_counts()
df_main['label'].value_counts(normalize = True)
# Normalizing the data means converting all values fell between 0 and 1 BUT were NOT 0 or 1.

0    0.750175
1    0.249825
Name: label, dtype: float64

The data is unbalanced, so I use sampling to avoid the affect from imbalancing dataset

In [17]:
X = X.values
# Extract all values and convert into series object (in an array format)

# MAKE MODELLING FUNCTION

When we build this function we use some method :
1. Training and testing split
2. Stratified 10-Fold Cross Validation each method classification in training data
3. Balancing data when we build a model, but in validation fold is similar like original data
4. Evaluate every model

In [18]:
import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import make_scorer,accuracy_score,roc_auc_score,precision_score,recall_score,f1_score,log_loss

def geometric_mean_pres_rec(y_true,y_pred):
    score = precision_score(y_true,y_pred)*recall_score(y_true,y_pred)
    return(score)
# This gives you a score, which is the model's performance (in terms of precision and recall)

# model_classification, when called runs all these classification methods (all 7)
def model_classification(verbose):
    # simple classification
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.naive_bayes import BernoulliNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    
    # hard classification
    from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
    
    if verbose:
        print("Set Simple Model")
    logreg = LogisticRegression(penalty='l2')
    knn = KNeighborsClassifier()
    nb = BernoulliNB()
    dt = DecisionTreeClassifier()
    
    if verbose:print("Set Hard Model\n")
    rf = RandomForestClassifier(random_state = 77)
    gbc = GradientBoostingClassifier(random_state=77 )
    xgb = XGBClassifier(random_state=77)
    
    model = [logreg,knn,nb,dt,rf,gbc,xgb]
    method_name = ['Logistic Regression',
                  'K-Nearest Neighbor',
                  'Naive Bayes',
                  'Decision Tree',
                  'Random Forest',
                  'Gradient Boosting Classifier',
                  'Extreme Gradient Boosting']
    return [model,method_name]
    
def pipe_imbalance(X,y,imb = RandomUnderSampler(),verbose = False):
# This takes two arguments (the features and the label from earlier) and run all these things in the column on them. 
# Basically, it just evaluates all of them on these metrics    
    df_eval = pd.DataFrame(columns = ['Model',
                                    'Accuracy',
                                    'Precision',
                                    'Recall',
                                    'AUC',
                                    'F1_score',
                                    'Log_loss',
                                    'Geometric_Mean',
                                    'Geometric_Mean_Precision_Recall',
                                    'Time'])

    if verbose: print("Split Training and Testing\n")
    X_train, X_test,y_train,y_test = train_test_split(X,
                                                        y,
                                                        test_size = 0.3,
                                                        stratify = y,
                                                        #random_state = 123
                                                     )

    if verbose: print('Import Classification Method\n')
    
    list_model = model_classification(True)
    df_eval['Model']=list_model[1]

    if verbose: print('Building Pipeline\n')
    pipe = Pipeline([('imb',imb),('classifier',LogisticRegression())])

    if verbose: print('Defining Params and Scoring\n')
    params = {'classifier': list_model[0]}
    scorers = {'accuracy':make_scorer(accuracy_score),
               'precision':make_scorer(precision_score),
               'recall':make_scorer(recall_score),
               'roc_auc':make_scorer(roc_auc_score),
               'f1':make_scorer(f1_score),
               'neg_log_loss':'neg_log_loss',
               'gm':make_scorer(geometric_mean_score),
               'gmpr':make_scorer(geometric_mean_pres_rec)}
    
    skf = StratifiedKFold(n_splits=10,random_state = 7)
    
    grid = GridSearchCV(estimator = pipe,
                    param_grid = params,
                    scoring = scorers,
                    refit = 'accuracy',
                    cv = skf)
    grid.fit(X_train,y_train)
    
    df_eval['Accuracy']  = grid.cv_results_['mean_test_accuracy']
    df_eval['Precision'] = grid.cv_results_['mean_test_precision']
    df_eval['Recall'] = grid.cv_results_['mean_test_recall']
    df_eval['AUC'] = grid.cv_results_['mean_test_roc_auc']
    df_eval['F1_score'] = grid.cv_results_['mean_test_f1']
    df_eval['Log_loss'] = grid.cv_results_['mean_test_neg_log_loss']
    df_eval['Geometric_Mean'] = grid.cv_results_['mean_test_gm']
    df_eval['Geometric_Mean_Precision_Recall'] = grid.cv_results_['mean_test_gmpr']
    df_eval['Time'] = grid.cv_results_['mean_fit_time']
    
    return [grid,df_eval,X_test,y_test]


In [19]:
df_under = pipe_imbalance(X,y, verbose=True)

Split Training and Testing

Import Classification Method

Set Simple Model
Set Hard Model

Building Pipeline

Defining Params and Scoring



In [20]:
df_under[1]

,Model,Accuracy,Precision,Recall,AUC,F1_score,Log_loss,Geometric_Mean,Geometric_Mean_Precision_Recall,Time
0,Logistic Regression,0.249857,0.249857,1.000000,0.500000,0.399817,-0.703113,0.000000,0.249857,0.044190
1,K-Nearest Neighbor,0.653369,0.376148,0.586405,0.631039,0.458130,-1.337362,0.629260,0.220570,0.069326
2,Naive Bayes,0.508423,0.252909,0.494450,0.503766,0.334137,-0.694608,0.501920,0.125272,0.018042
3,Decision Tree,0.576507,0.314033,0.587264,0.580094,0.409172,-14.624808,0.579796,0.184789,0.303095
4,Random Forest,0.659079,0.383875,0.598977,0.639039,0.467650,-0.619364,0.637518,0.230305,3.306624
5,Gradient Boosting Classifier,0.670141,0.396895,0.615548,0.651938,0.482390,-0.598009,0.650599,0.244461,6.997249
6,Extreme Gradient Boosting,0.645875,0.374840,0.624679,0.638807,0.468460,-0.665347,0.638488,0.234452,1.867273


In [21]:
df_smote = pipe_imbalance(X,y,imb=SMOTE(k_neighbors=11), verbose=True)

Split Training and Testing

Import Classification Method

Set Simple Model
Set Hard Model

Building Pipeline

Defining Params and Scoring



From the model comparison dataframe, we can see that Gradient boosting Classifier with Random Under Sampling is the best Model because It has good evaluation metrics and the highest AUC Score. After we get the model, We tune the Gradient boosting Classifier parameters.

# Tuning Hyperparameter

In this section, we will optimize our model by tuning our hyperparameter. The model that will be tuned is Gradient Boosting Classifier. We use this model beacuse it has good evaluation metrics among the other model. The parameter that will be tuned are:
1. Number of Tree (n_estimators)
2. Maximum of Tree (max_depth)
3. Minimum Sample in Leaf (min_samples_leaf)
4. Minimum Sample in each split (min_samples_split)
5. Number of Data for Bootstrap (subsample)
6. Learning Rate (learning_rate)

In [22]:
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter 

# EVAL METRICS

In [23]:
# Library for Evaluation Metrics
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import make_scorer,precision_score
from sklearn.metrics import recall_score,accuracy_score,f1_score

# library for calculate the cross validation score
from sklearn.model_selection import cross_val_score
 
# library for Stratified Cross Validation
from sklearn.model_selection import StratifiedKFold

In [24]:
def geometri_score(y_true,y_pred):
    from sklearn.metrics import confusion_matrix
    spe = confusion_matrix(y_true,y_pred)[0,0]/(confusion_matrix(y_true,y_pred)[0,0]+confusion_matrix(y_true,y_pred)[0,1])
    sen = recall_score(y_true,y_pred)
    acc = accuracy_score(y_true,y_pred)
    eval_baru = (spe*sen*acc)**(1/3)
    return eval_baru

geometri = make_scorer(geometri_score)

In [25]:
from sklearn.metrics import make_scorer,precision_score,recall_score,accuracy_score,auc
scorers = {
    'precision_score':make_scorer(precision_score),
    'recall_score':make_scorer(recall_score),
    'accuracy_score':make_scorer(accuracy_score),
    'auc':make_scorer(auc),
    'geometric_mean_score': make_scorer(geometri_score)
}
print(scorers)

{'precision_score': make_scorer(precision_score), 'recall_score': make_scorer(recall_score), 'accuracy_score': make_scorer(accuracy_score), 'auc': make_scorer(auc), 'geometric_mean_score': make_scorer(geometri_score)}


In [26]:
# Develop Evaluation Function

skf = StratifiedKFold(n_splits=5,random_state=10)

def eval_cv(alg,X,y):
    print("Accuracy cv : \n",
          cross_val_score(alg,X,y,scoring="accuracy",cv=skf),"\n",
          cross_val_score(alg,X,y,scoring="accuracy",cv=skf).mean(),"\n")
    print("Recall cv : \n",
          cross_val_score(alg,X,y,scoring="recall",cv=skf),"\n",
          cross_val_score(alg,X,y,scoring="recall",cv=skf).mean(),"\n")
    print("Precision cv : \n",
          cross_val_score(alg,X,y,scoring="precision",cv=skf),"\n",
          cross_val_score(alg,X,y,scoring="precision",cv=skf).mean(),"\n")
    print("f1 cv : \n",
          cross_val_score(alg,X,y,scoring="f1",cv=skf),"\n",
          cross_val_score(alg,X,y,scoring="f1",cv=skf).mean(),"\n")

    
def eval(alg,X,y):
    alg.fit(X_train,y_train)
    print("Accuracy:\n",
          accuracy_score(y,alg.predict(X)),"\n")
    print("Confusion matrix test:\n",
          confusion_matrix(y,alg.predict(X)),"\n")
    print("Classification Report test:\n",
          classification_report(y,alg.predict(X)),"\n")

### Train-test Split and Modelling (Default)

In [27]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
from numpy import loadtxt
from numpy import sort
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, stratify=y)
rus = RandomUnderSampler()
X_sampled,y_sampled = rus.fit_sample(X_train,y_train)
gbc = GradientBoostingClassifier()
model=gbc.fit(X_sampled,y_sampled)

Wall time: 7.11 s


In [28]:
gbc.get_params

<bound method BaseEstimator.get_params of GradientBoostingClassifier()>

In [29]:
gbc_tune = GradientBoostingClassifier(criterion='friedman_mse',
                    learning_rate=0.1,
                    max_depth=3,
                    max_features=None,
                    min_samples_split=2,
                    min_samples_leaf=1,
                    n_estimators=100,
                    subsample=1.0,                  
                    random_state=123)

pipe_gbc_tune = Pipeline([('imb',RandomUnderSampler(random_state=123)),('clf',gbc_tune)])

In [30]:
eval_cv(pipe_gbc_tune,X_train,y_train)

Accuracy cv : 
 [0.68355334 0.68283981 0.67380443 0.66702355 0.66952177] 
 0.6753485800749325 

Recall cv : 
 [0.60912981 0.59714286 0.61857143 0.64142857 0.61      ] 
 0.6152545343386998 

Precision cv : 
 [0.41057692 0.40780488 0.40092593 0.39699381 0.39537037] 
 0.4023343816417828 

f1 cv : 
 [0.49052269 0.48463768 0.48651685 0.49044238 0.47977528] 
 0.4863789770627228 



# Tuning 1 (n_estimators)

In [ ]:
rus = RandomUnderSampler(random_state=123)

gbc_tune = GradientBoostingClassifier(criterion='friedman_mse',
                    learning_rate=0.1,
                    max_depth=3,
                    max_features=None,
                    min_samples_split=2,
                    min_samples_leaf=1,
#                    n_estimators=100,
                    subsample=1.0,                  
                    random_state=123)

select = SelectFromModel(gbc_tune,threshold='median')
pipe_gbc = Pipeline([('rus',rus),('select',select),('clf',gbc_tune)])

param_gb = {
    'clf__n_estimators':[45,70,90,100,110,120,140]}
# It will run for all these values. It decided 45 was the best.

skf = StratifiedKFold(n_splits=5,random_state=123)

gbc_grid = GridSearchCV(pipe_gbc,
                       param_grid=param_gb,
                       # Tuning Number 1/4
                       refit = 'geometric',
                       # Method Validation
                       cv=skf)

gbc_grid.fit(X_train,y_train)

In [ ]:
print(gbc_grid.best_params_)
print(gbc_grid.best_score_)
eval(gbc_grid.best_estimator_,X_test,y_test)

# Tuning 2 ( Max_depth & Min_samples_split)

In [ ]:
rus = RandomUnderSampler(random_state=123)

# Tune  the models (minimum number of samples for max depth)
gbc_tune2 = GradientBoostingClassifier(criterion='friedman_mse',
                    learning_rate=0.1,
#                    max_depth=3,
                    max_features=None,
#                    min_samples_split=2,
                    min_samples_leaf=1,
                    n_estimators=100,
                    subsample=1.0,                  
                    random_state=123)

select = SelectFromModel(gbc_tune2,threshold='median')
pipe_gbc2 = Pipeline([('rus',rus),('select',select),('clf',gbc_tune2)])

# parameter tuning
param_gb2 = {
    'clf__max_depth':range(2,8,1),
    'clf__min_samples_split':range(2,6,2)
# Total 8 parameters. 
}

skf = StratifiedKFold(n_splits=5,random_state=123)

gbc_grid2 = GridSearchCV(pipe_gbc2,
                       param_grid=param_gb2,
                       # GridSearchCV tuning number 2
                       refit = 'geometric',
                       # Method Validation
                       cv=skf)

gbc_grid2.fit(X_train,y_train)

In [ ]:
print(gbc_grid2.best_params_)
print(gbc_grid2.best_score_)
eval(gbc_grid2.best_estimator_,X_test,y_test)

# Tuning 3 (Min_samples_leaf)

In [ ]:
rus = RandomUnderSampler(random_state=123)

gbc_tune3 = GradientBoostingClassifier(criterion='friedman_mse',
                    learning_rate=0.1,
#                    max_depth=3,
                    max_features=None,
                    min_samples_split=2,
#                    min_samples_leaf=1,
                    n_estimators=100,
                    subsample=1.0,                  
                    random_state=123)

select = SelectFromModel(gbc_tune3,threshold='median')
pipe_gbc3 = Pipeline([('rus',rus),('select',select),('clf',gbc_tune3)])

param_gb3 = {
    'clf__max_depth':[3,4,5],
    'clf__min_samples_leaf':[1,2,3,4,5,6]
}

skf = StratifiedKFold(n_splits=5,random_state=123)

gbc_grid3 = GridSearchCV(pipe_gbc3,
                       param_grid=param_gb3,
                       # GridSearchCV tuning number 3
                       refit = 'geometric',
                       # Method Validation
                       cv=skf)

gbc_grid3.fit(X_train,y_train)

In [ ]:
print(gbc_grid3.best_params_)
print(gbc_grid3.best_score_)
eval(gbc_grid3.best_estimator_,X_test,y_test)

# Tuning 4 (Learning Rate)
# Trying to see what learning rate gives the fastest output. If the learning rate is too small, it takes very long. If too big it also takes too long as well

In [ ]:
rus = RandomUnderSampler(random_state=123)

# model yang disiapkan
gbc_tune4 = GradientBoostingClassifier(criterion='friedman_mse',
#                    learning_rate=0.1,
                    max_depth=3,
                    max_features=None,
                    min_samples_split=2,
                    min_samples_leaf=1,
                    n_estimators=100,
                    subsample=1.0,                  
                    random_state=123)

select = SelectFromModel(gbc_tune4,threshold='median')
pipe_gbc4 = Pipeline([('rus',rus),('select',select),('clf',gbc_tune4)])

param_gb4 = {
    'clf__learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,1.5,2]
}

skf = StratifiedKFold(n_splits=5,random_state=123)

gbc_grid4 = GridSearchCV(pipe_gbc4,
                       param_grid=param_gb4,
                       refit = 'geometric',
                       cv=skf)

gbc_grid4.fit(X_train,y_train)

In [ ]:
print(gbc_grid4.best_params_)
print(gbc_grid4.best_score_)
eval(gbc_grid4.best_estimator_,X_test,y_test)

# Model Fix

## The Final Model that will be used in this dataset is Gradient Boosting Classifier with Random Under sampling and Stratified 10-Cross Validation with Parameter Tuning. I have made the pipeline for this final model 

In [ ]:
rus = RandomUnderSampler(random_state=123)

# model yang disiapkan
gbc_tune4 = GradientBoostingClassifier(criterion='friedman_mse',
                    learning_rate=0.1,
                    max_depth=3,
                    max_features=None,
                    min_samples_split=2,
                    min_samples_leaf=1,
                    n_estimators=100,
                    subsample=1.0,                  
                    random_state=123)

select = SelectFromModel(gbc_tune4,threshold='median')
pipe_gbc4 = Pipeline([('rus',rus),('clf',gbc_tune4)])

In [ ]:
pipe_gbc4.fit(X,y)

In [ ]:
y_prob = pipe_gbc4.predict_proba(X_test)

# Metrics Evaluation

## AUC-ROC Curve

In [ ]:
from sklearn.metrics import roc_curve

def plot_roc_curve(ytest, P_ensemble):
    """Plot the roc curve for base learners and ensemble."""
    plt.figure(figsize=(10, 8))
    plt.plot([0, 1], [0, 1], 'k--')

    fpr, tpr, _ = roc_curve(y_test, P_ensemble)
    plt.plot(fpr, tpr)
        
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.show()

In [ ]:
plot_roc_curve(y_test,y_prob[:,1])

## Confusion Matrix

In [ ]:
y_pred = pipe_gbc4.predict(X_test)
print(classification_report(y_test,y_pred))

# Feature Importance

### List from Feature that has big affect to model prediction. Feature that affect whether the drivers was dangerous or not

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
from numpy import loadtxt
from numpy import sort
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, stratify=y)
rus = RandomUnderSampler()
X_sampled,y_sampled = rus.fit_sample(X_train,y_train)
gbc = GradientBoostingClassifier()
model=gbc.fit(X_sampled,y_sampled)

In [ ]:
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]


In [ ]:
to_bar = pd.DataFrame([df_main.columns[indices],importances[indices]]).transpose()
to_bar.columns = ['Feature','Value']

In [ ]:
#df_main.info()

In [ ]:
# Print the feature ranking
print("Feature ranking:")

for f in range(20):
    print("%d. feature %s (%f)" % (f + 1, df_main.columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest

plt.figure(1, figsize=(10, 15))
plt.title("Feature importances")
ax = sns.barplot(x='Value',y='Feature',data=to_bar[0:30])
plt.xlabel('Importances Value', fontsize = 20)
plt.ylabel('Feature', fontsize = 20)
plt.show()

# Radian x, then distance is high and it affects the most. This is the graph below.
